In [ ]:
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python_operator import PythonOperator

# Define the DAG
dag = DAG(
    'example_xcom_schedule',
    default_args={
        'owner': 'airflow',
        'retries': 1,
        'retry_delay': timedelta(minutes=5),
    },
    description='An example DAG demonstrating how to use XComs to pass data between tasks.',
    schedule_interval=timedelta(days=1),
    start_date=datetime(2024, 4, 22),
)

# Task 1: Generate a parameter
def generate_parameter(**context):
    parameter_value = 42  # Example parameter value
    # Return the parameter value
    return parameter_value

generate_parameter_task = PythonOperator(
    task_id='generate_parameter',
    python_callable=generate_parameter,
    provide_context=True,
    dag=dag,
)

# Task 2: Use the parameter from Task 1
def use_parameter(**context):
    # Fetch the parameter from the previous task's XCom
    ti = context['ti']
    parameter_value = ti.xcom_pull(task_ids='generate_parameter')
    
    # Use the parameter in this task
    print(f"Parameter received from Task 1: {parameter_value}")
    # Perform the necessary task with the parameter
    
use_parameter_task = PythonOperator(
    task_id='use_parameter',
    python_callable=use_parameter,
    provide_context=True,
    dag=dag,
)

# Define the task dependencies
generate_parameter_task >> use_parameter_task
